In [ ]:
import numpy as np
import scipy as spy
import scipy.integrate as integrate
import scipy.special as special
#from sympy import *
import matplotlib.pyplot as plt
#from py_functions import getalpha0

In [ ]:
# Function for calculating force and energy with bending for given actuator
def BendingForce(lp,t,w,nu,Ymm,alp,lamb,A): 
    D = ((t**3)*w*Ymm)/(12*(1-nu)**2) # Bending stiffness
    k = (2*alp)/lp # Curvature of membrane

    Ub = D*k**2*lp # Bending Energy

    # Force w/ Bending
    Fwb = lamb*(np.cos(alp)/(1-np.cos(alp))) + 2*D*(alp-np.sin(alp)*np.cos(alp)/A) * (np.cos(alp)/(1-np.cos(alp))) * (1+(2*alp*np.tan(alp)**2)/(alp-np.tan(alp)))
    return Fwb, Ub

# function gets intital tip angle
def alp0get(Eper):
    alpeqn = ((1-Eper)**2)*(2/np.pi) # Easy to calc constant
    y = 0.01 # intial guess of angle
    it = 0.0001 # iteration
    scale = 1.6/it # how many possible
    end = int(scale) # int for loop
    for i in range(0,end): # over scale
        
        func = (y-np.sin(y)*np.cos(y))/(y**2) - alpeqn # Find value of remaining equation with embedded alpha0
        
        if func >= 0: # root is transition from negative to positive in this case
            alp0 = y - it # previous value before sign flip returned       
            break #root found, stop
        else:
            y += it #move to next guess
    return alp0

# function defines lamb(da) for easy force calcs        
def lamb(E,perm0,permr,w,t):
    return lamb = E**2*perm0*permr*w*t

# function sets up intial actuator geometry
def actgeometry(Lp,Eper,alp0,lamb,alpf):
    # Electrode length
    Le = Lp*Eper

    #Height
    h = Lp*(np.sin(alp0)/alp0) # Height [m]

    #Area w/ restraints 
    # !!! MIGHT NOT WORK WITH ARRAY !!!
    A = (Lp - Le)**2 /np.pi # Area [m2]

    #Blocking force
    Fb = (lamb)*(np.cos(alp0)/(1-np.cos(alp0)))

    Lpf = Lp - Le # Fully Deformed pouch length 

    xmax = h - (Lpf*(np.sin(alpf)/alpf)) - Le # max displacement
    return Le,h,A,Fb,Lpf,xmax

# function evolves acutator
def evolveact(A,alp,Lp,h,k):
    # Blocks of lp(alp) eqn sweep
    lpnum = 2*A*np.square(alp)
    lpdom = (alp - np.sin(alp)*np.cos(alp))
    lp2 = lpnum/lpdom
    lp = lp2**0.5
    lp[0] = Lp # Removes unphysical discretisation error

    # electrode length sweep
    le = Lp - lp 

    # displacement sweep
    lpmp = np.sin(alp)/alp # lp multiplier in x
    lplpmp = lp*lpmp
    X = h - lplpmp - le

    # Load of spring
    P = k*X

In [ ]:
## Model Constants
resolution = 1000 # res of thing to be swept
sweepmax = 0.03
step = Lpmax/resolution

# variable of interest
Lp = np.arange(step,sweepmax,step) # [m] Initial Pouch Length - From original kellaris paper (2018)

perm0 = 8.85418782e-12#[m-3kg-1s4A2]
permr = 2.2 # for BOPP
Eb = 700e6 #[V/m] the breakdown strength of BOPP
w = 0.01 #[m] from original kellaris paper
t = 18e-6 #[m] Bopp thickness used in initial designs
k = 100000 #[Nm-1] Achilles tendon average sourced from Litchwark 2005
rho_f = 903 #[kgm-3] Bopp Density TEKRA (2020)
rho_o = 920 #[kgm-3] FR3 Dielectric oil Density Cargill (2018)
alpf = np.pi/2 #[rad] Assume circular cross section on stroke completion
Eper = 0.5 # Percentage of electrode coverage
Va = 10e3 # Actuation voltage
jj = 1 # Array Scaling